In [0]:
!pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.52-py2.py3-none-any.whl size=21997 sha256=cad8d4636dcaa42e5ad2ba744ef61e2fdaab51e38a9fc347e3fa2f2c2d9e3ac4
  Stored in directory: /root/.cache/pip/wheels/1b/9d/ce/929e851fc7d649cd962c177220ad985675785aa97c54df5714
Successfully built yfinance


In [0]:
!pip install pyflux

     |████████████████████████████████| 1.3MB 5.0MB/s 
     |████████████████████████████████| 962kB 42.7MB/s 
  Created wheel for pyflux: filename=pyflux-0.4.15-cp36-cp36m-linux_x86_64.whl size=4656534 sha256=813098032a8a5e5ba24bbbc0125508e79877f388bc9d88bcbe38f2c8ecbf6d20
  Stored in directory: /root/.cache/pip/wheels/36/24/09/d18fa4f926cc90598d23ebd09fb4b0fd7b3dd1d2f506b952de
Successfully built pyflux


In [0]:
import numpy as np
import pandas as pd
import pyflux as pf
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
%matplotlib inline


In [0]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
factor_matrix = np.loadtxt('/content/drive/My Drive/factor_mat.csv')
factor_matrix = factor_matrix.T
factor_matrix.shape

(29325, 77)

In [0]:
company = pd.read_csv("/content/drive/My Drive/company_table1.csv")
stock_ID = company['stock_ID'].tolist()
private_indices = [i for i, x in enumerate(stock_ID) if x == "private"]
stock_ID = [x for i, x in enumerate(stock_ID) if x != "private"]
new_factor = np.delete(factor_matrix, private_indices, 1)
new_factor.shape

(29325, 69)

In [0]:
news_guardian_entities = pd.read_csv('/content/drive/My Drive/news_guardian_cuttedentity.csv')
df_factor = pd.DataFrame(data=new_factor, index=pd.to_datetime(news_guardian_entities.timestamp).dt.date, columns=stock_ID)

In [0]:
company = list(df_factor.columns)
company.remove('RDS.A')
company.remove('[005930]')
company.remove('LNKD (2016)')
company.remove('4938')

In [0]:
company[0]

'EBAY'

In [0]:
arimax_list = []
arima_list = []

for name in company:
  FB_sentiment = df_factor.groupby('timestamp')[name].mean().reset_index()
  data = yf.download(name, start="2017-09-28", end="2019-10-02")

  count = 0
  irow = 0
  while irow < data.shape[0] - 1:
    next_date = data.iloc[irow,:].name + timedelta(days=1)
    if next_date not in data.index:
      count += 1
      data.loc[next_date] = data.iloc[irow, :]
      data = data.sort_index()
    irow += 1
  data_sorted = data.sort_index()
  #create a new column for return 
  data_sorted.insert(6, 'Return', 1)
  for i in range(1, data_sorted.shape[0]):
    return_val = data_sorted.iloc[i-1, 4] / data_sorted.iloc[i, 4] - 1
    data_sorted.iloc[i, 6] = return_val
  data_sorted = data_sorted.iloc[3:, :]
  FB_sentiment = FB_sentiment.set_index("timestamp")
  final_FB = data_sorted.join(FB_sentiment)
  final_FB = final_FB.iloc[:,6:]
  final_FB = final_FB.fillna(0)
  plt.figure(figsize=(15,5));
  plt.plot(final_FB.index, final_FB['Return']);
  plt.ylabel('Return');
  plt.title('Return for ' + name)
  plt.plot();

  arima_pred = []
  arimax_pred = []

  train_size = int(final_FB.shape[0] * 0.80)
  test_original = final_FB.iloc[train_size:final_FB.shape[0],:]
  test_size = final_FB.shape[0] - train_size
  test = final_FB.iloc[train_size:final_FB.shape[0],:]
  train = final_FB.iloc[0:train_size,:]
  model = pf.ARIMAX(data=final_FB, formula='Return~1',
              ar=5, ma=5, integ = 5, family=pf.Normal())
  x = model.fit('MLE')
  model.plot_fit(figsize=(15,10))
  a_pred = model.predict_is(10, False)
  
  arima_pred = a_pred.values

  model_x = pf.ARIMAX(data=final_FB, formula='Return~1+'+name,
              ar=5, ma=5, integ = 5, family=pf.Normal())

  ax_pred = model_x.predict_is(10, False)
  model_x.plot_predict_is(h=10, fit_once = False, figsize=(15,10)) 
  arimax_pred = ax_pred.values

  x = model_x.fit('MLE')
  model_x.plot_fit(figsize=(15,10))

  MSE = mean_squared_error(train.Return.values[-10:], arima_pred[-10:])
  arima_list.append(MSE)
  MSE_x = mean_squared_error(train.Return.values[-10:], arimax_pred[-10:])
  arimax_list.append(MSE_x)
  print('MSE from ARIMA for ', name, ': ', round(MSE, 5))
  print('MSE from ARIMAX for ', name, ': ', round(MSE_x, 5))

In [0]:
res = pd.DataFrame({"ARIMA":arima_list, "ARIMAX":arimax_list}, index = company)

In [0]:
from google.colab import files
res.to_csv('ARIMAX.csv')
files.download('ARIMAX.csv')

In [0]:
finall = res[res['ARIMA']>=res['ARIMAX']]

In [0]:
finall.to_csv("ARIMAX.csv")